In [1028]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [1029]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

workers - workers.csv


,worker_id,name,плоская,высокая,струйная
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


equipment - equipment.csv


,machine_id,machine_type,operators_needed
0,PM-01,плоская,4
1,PM-02,плоская,4
2,PM-03,высокая,4
3,PM-04,высокая,4
4,SM-01,струйная,1


schedule - schedule_template.csv


,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


requirements - position_requirements.csv


,machine_type,position,min_rank,profession_required
0,плоская,1,7,Печатник
1,плоская,2,6,Печатник
2,плоская,3,5,Печатник
3,плоская,4,4,Печатник
4,струйная,1,5,Оператор струйной печати


plan - plan.csv


,machine_id,night,day,evening
0,PM-01,True,True,True
1,PM-02,True,True,True
2,PM-03,True,True,True
3,PM-04,True,True,True
4,SM-01,True,True,True


In [1030]:
# Определяем основную профессию
cols = ["плоская", "высокая", "струйная"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,W001,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,W002,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,W003,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,W004,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,W005,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [1031]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
target_shift = "День"
shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}

In [1032]:
# Функция для переворота смены
def shift_next_week(week, shift_name):
    shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}
    print(f"shift_workers - Кандидаты для работы в смену: {shift_name} недели {week}")
    result = schedule[
        (schedule["week"] == week - 1)
        &
        # (schedule["day"] == target_day) &
        (schedule["shift"] == shift[shift_name])
    ]
    result = result.merge(workers, on="worker_id", how="left")
    print(f"В смене работает {len(result)} человек")

    return result

In [1045]:
shift_workers_day = shift_next_week(target_week, "День")
display(shift_workers_day.head())

shift_workers_evning = shift_next_week(target_week, "Вечер")
# display(shift_workers_evning)

shift_workers_nigth = shift_next_week(target_week, "Ночь")
# display(shift_workers_nigth)

shift_workers - Кандидаты для работы в смену: День недели 2
В смене работает 20 человек


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,1,Вечер,W021,Работник П 21,7,6,0,плоская,"[плоская, высокая]"
1,1,Вечер,W022,Работник П 22,6,5,0,плоская,"[плоская, высокая]"
2,1,Вечер,W023,Работник П 23,5,4,0,плоская,"[плоская, высокая]"
3,1,Вечер,W024,Работник П 24,4,0,5,струйная,"[плоская, струйная]"
4,1,Вечер,W025,Работник П 25,7,6,0,плоская,"[плоская, высокая]"


shift_workers - Кандидаты для работы в смену: Вечер недели 2
В смене работает 20 человек
shift_workers - Кандидаты для работы в смену: Ночь недели 2
В смене работает 20 человек


In [1034]:
def build_shift_rotation(schedule: pd.DataFrame, workers: pd.DataFrame, target_week: int) -> pd.DataFrame:
    """
    Формирует общий датафрейм кандидатов на target_week для всех смен сразу,
    применяя правило переворота смен:
        Ночь -> День, День -> Вечер, Вечер -> Ночь
    """
    shift_map = {"Ночь": "Вечер", "День": "Ночь", "Вечер": "День"}

    # Базовый слой — прошлая неделя
    prev = schedule.loc[schedule["week"] == target_week - 1].copy()

    # Сохраним прошлую смену (на всякий случай для анализа)
    prev = prev.rename(columns={"shift": "prev_shift"})

    # Рассчитаем смену на следующую неделю и проставим неделю
    prev["shift"] = prev["prev_shift"].map(shift_map)
    prev["week"] = target_week

    # Объединим с данными по работникам
    result = prev.merge(workers, on="worker_id", how="left")

    # Короткая статистика по сменам
    counts = result["shift"].value_counts()
    total = int(counts.sum())
    print(f"Кандидаты на работу в неделю {target_week}: всего {total}")
    for s in ["День", "Вечер", "Ночь"]:
        print(f"  {s}: {int(counts.get(s, 0))}")

    return result


In [1035]:
shift_workers_all = build_shift_rotation(schedule, workers, target_week)
display(shift_workers_all.head())

# # При необходимости — получить конкретную смену:
# shift_workers_day   = shift_workers_all.query('shift == "День"')
# shift_workers_even  = shift_workers_all.query('shift == "Вечер"')
# shift_workers_night = shift_workers_all.query('shift == "Ночь"')


Кандидаты на работу в неделю 2: всего 60
  День: 20
  Вечер: 20
  Ночь: 20


,week,prev_shift,worker_id,shift,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,2,Ночь,W001,Вечер,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,2,Ночь,W002,Вечер,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,2,Ночь,W003,Вечер,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,2,Ночь,W004,Вечер,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,2,Ночь,W005,Вечер,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [1036]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None
    
    return shift_equipment

In [1037]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# Заменяем названия смен на русские
plan_long = plan_long.replace(
    {"shift": {"night": "Ночь", "day": "День", "evening": "Вечер"}}
)

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week

# Позже убрать дублирование кода
shift_equipment = f_shift_equipment(plan_long, target_shift)

# Создаем расписание для каждой смены с пустыми позициями работников
shift_equipment_day = f_shift_equipment(plan_long, 'День')
shift_equipment_evening = f_shift_equipment(plan_long, 'Вечер')
shift_equipment_night = f_shift_equipment(plan_long, 'Ночь')


display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,None
1,2,День,PM-01,плоская,2,6,Печатник,None
2,2,День,PM-01,плоская,3,5,Печатник,None
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,None
1,2,Вечер,PM-01,плоская,2,6,Печатник,None
2,2,Вечер,PM-01,плоская,3,5,Печатник,None
3,2,Вечер,PM-01,плоская,4,4,Печатник,None
4,2,Вечер,PM-02,плоская,1,7,Печатник,None


In [1038]:
def candidat(s, profesion, min_rank, shift_name):
    # Базовая маска: работники в нужной смене и не назначенные ранее
    base_mask = (shift_workers_all["shift"] == shift_name) & (
        ~shift_workers_all["worker_id"].isin(appointed)
    )
    
    # Общая маска для "second" режима: профессия есть в списке всех профессий
    all_professions_mask = base_mask & (
        shift_workers_all["все_профессии"].apply(lambda x: profesion in x)
    )

    if s == "ferst":
        # Режим "ferst": ищем основной персонал с точным рангом
        candidates = shift_workers_all[
            base_mask
            & (shift_workers_all[profesion] == min_rank)
            & (shift_workers_all["основная_профессия"] == profesion)
        ]
    elif s == "second":
        # Режим "second": ищем персонал с точным min_rank ИЛИ min_rank + 1
        # Используем маску all_professions_mask для уменьшения дублирования
        rank_conditions = (shift_workers_all[profesion] == min_rank) | (
            shift_workers_all[profesion] == min_rank + 1
        )
        
        candidates = shift_workers_all[all_professions_mask & rank_conditions]
    elif s == "third":
        # Режим "third": ищем персонал c любым рангом
        # Используем маску all_professions_mask для уменьшения дублирования
        candidates = shift_workers_all[all_professions_mask]
    else:
        # Обработка неверного значения параметра 's'
        print(f"Ошибка: Неизвестный режим '{s}'. Используйте 'ferst' или 'second'.")
        candidates = pd.DataFrame() # Возвращаем пустой DataFrame

    return candidates


def position_assignment(shift_equipment, s="ferst", shift_name="День"):
    # appointed = set()
    free_machin_positions = []

    for index, row in shift_equipment.iterrows():
        if not row["worker_id"]:
            # print(row["worker_id"], "->", end=" ")
            profesion = row["machine_type"]
            min_rank = row["min_rank"]

            candidates = candidat(s, profesion, min_rank, shift_name)

            # candidates = shift_workers_day[
            #     (shift_workers_day["основная_профессия"] == profesion)
            #     & (shift_workers_day[profesion] == min_rank)
            #     & (~shift_workers_day["worker_id"].isin(appointed))
            # ]
            # print(candidates)

            if not candidates.empty:
                first = candidates.iloc[0]  # <-- первая строка
                worker_id = first["worker_id"]

                shift_equipment.loc[index, "worker_id"] = worker_id  # <-- loc, не iloc
                appointed.add(worker_id)
                # print(worker_id)
            else:
                # при необходимости логировать отсутствие кандидатов
                # print(f"Нет кандидатов для index={index}, {profesion}, min_rank={min_rank}")
                free_machin_positions.append(shift_equipment.loc[index])
        else:
            # print(row["worker_id"], "уже назначен")
            pass

    return pd.DataFrame(free_machin_positions), shift_equipment


appointed = set()

In [1039]:
def run_assignment_for_shift(shift_equipment, default_tourse):
    # Первый тур назначения - основная профессия, минимальный ранг
    free_machin_positions, shift_equipment = position_assignment(shift_equipment, default_tourse[0][0], default_tourse[0][1])

    if not free_machin_positions.empty:
        print(f"Остались свободные позиции оборудования после тура 1 назначения:")
        for i in range(5):
            profession, shift_name = default_tourse[i + 1]
            # print(f"Остались свободные позиции оборудования после тура {i + 1} назначения:")
            # display(free_machin_positions)
            free_machin_positions, shift_equ = position_assignment(free_machin_positions, profession, shift_name)
            shift_equipment = shift_equipment.combine_first(shift_equ)
            if free_machin_positions.empty:
                break
            print(f"Остались свободные позиции оборудования после тура {i + 2} назначения:")
            # display(free_machin_positions)
    return shift_equipment

In [1040]:
default_tourse = [
    ("ferst",  "День"),
    ("second", "День"),
    ("ferst",  "Ночь"),
    ("second", "Ночь"),
    ("ferst",  "Вечер"),
    ("second", "Вечер"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

shift_equipment_day = run_assignment_for_shift(shift_equipment_day, default_tourse_day)
display(shift_equipment_day)
shift_equipment_evening = run_assignment_for_shift(shift_equipment_evening, default_tourse_evening)
display(shift_equipment_evening)
shift_equipment_night = run_assignment_for_shift(shift_equipment_night, default_tourse_night)
display(shift_equipment_night)

# shift_equipment_day

Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позиции оборудования после тура 5 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,W021
1,2,День,PM-01,плоская,2,6,Печатник,W022
2,2,День,PM-01,плоская,3,5,Печатник,W023
3,2,День,PM-01,плоская,4,4,Печатник,W037
4,2,День,PM-02,плоская,1,7,Печатник,W025
5,2,День,PM-02,плоская,2,6,Печатник,W026
6,2,День,PM-02,плоская,3,5,Печатник,W027
7,2,День,PM-02,плоская,4,4,Печатник,W038
8,2,День,PM-03,высокая,1,7,Печатник,W029
9,2,День,PM-03,высокая,2,6,Печатник,W030


Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позиции оборудования после тура 5 назначения:
Остались свободные позиции оборудования после тура 6 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,W005
1,2,Вечер,PM-01,плоская,2,6,Печатник,W002
2,2,Вечер,PM-01,плоская,3,5,Печатник,W003
3,2,Вечер,PM-01,плоская,4,4,Печатник,W043
4,2,Вечер,PM-02,плоская,1,7,Печатник,W041
5,2,Вечер,PM-02,плоская,2,6,Печатник,W006
6,2,Вечер,PM-02,плоская,3,5,Печатник,W007
7,2,Вечер,PM-02,плоская,4,4,Печатник,W047
8,2,Вечер,PM-03,высокая,1,7,Печатник,W009
9,2,Вечер,PM-03,высокая,2,6,Печатник,W010


Остались свободные позиции оборудования после тура 1 назначения:
Остались свободные позиции оборудования после тура 2 назначения:
Остались свободные позиции оборудования после тура 3 назначения:
Остались свободные позиции оборудования после тура 4 назначения:
Остались свободные позиции оборудования после тура 5 назначения:
Остались свободные позиции оборудования после тура 6 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,W045
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
5,2,Ночь,PM-02,плоская,2,6,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
8,2,Ночь,PM-03,высокая,1,7,Печатник,W049
9,2,Ночь,PM-03,высокая,2,6,Печатник,W050


In [1041]:
summary = (
    shift_equipment_night
    .groupby(["machine_id", "machine_type"], as_index=False)
    .agg(
        required=("position", "count"),
        assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum())
    )
)

# display(summary)

# Добавляем столбец "недостает"
summary["missing"] = summary["required"] - summary["assigned"]

# ---- 1. НЕПОЛНЫЕ БРИГАДЫ (частично укомплектованы)
incomplete = summary[
    (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
].copy()

incomplete

,machine_id,machine_type,required,assigned,missing
0,PM-01,плоская,4,1,3
2,PM-03,высокая,4,3,1
3,PM-04,высокая,4,3,1


In [1042]:
print(appointed)
print(len(appointed))

destaff = incomplete[incomplete['required'] / 2 >= incomplete['assigned']]['machine_id'].to_list()
# display(destaff)
release_workers = set()

for m_id in destaff:
    mask = (shift_equipment_night['machine_id'] == m_id) & (shift_equipment_night['worker_id'].notna())
    appointed -= set(w_id for w_id in shift_equipment_night[mask]['worker_id'].to_list())
    # убираем назначенных работников
    shift_equipment_night.loc[mask, 'worker_id'] = None

print(appointed)
print(len(appointed))

{'W048', 'W032', 'W002', 'W033', 'W053', 'W037', 'W014', 'W015', 'W036', 'W045', 'W013', 'W004', 'W050', 'W009', 'W010', 'W051', 'W043', 'W042', 'W006', 'W059', 'W060', 'W055', 'W025', 'W030', 'W028', 'W058', 'W021', 'W022', 'W012', 'W024', 'W018', 'W003', 'W007', 'W039', 'W001', 'W029', 'W011', 'W056', 'W026', 'W052', 'W017', 'W040', 'W005', 'W038', 'W023', 'W046', 'W034', 'W044', 'W057', 'W047', 'W027', 'W035', 'W016', 'W049', 'W019', 'W054', 'W031', 'W020', 'W041', 'W008'}
60
{'W048', 'W032', 'W002', 'W033', 'W053', 'W037', 'W014', 'W015', 'W036', 'W013', 'W004', 'W050', 'W009', 'W010', 'W051', 'W043', 'W042', 'W006', 'W059', 'W060', 'W055', 'W025', 'W030', 'W028', 'W058', 'W021', 'W022', 'W012', 'W024', 'W018', 'W003', 'W007', 'W039', 'W001', 'W029', 'W011', 'W056', 'W026', 'W052', 'W017', 'W040', 'W005', 'W038', 'W023', 'W046', 'W034', 'W044', 'W057', 'W047', 'W027', 'W035', 'W016', 'W049', 'W019', 'W054', 'W031', 'W020', 'W041', 'W008'}
59


In [1043]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_nigth[~shift_workers_nigth["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
4,1,День,W045,Работник П 45,7,6,0,плоская,"[плоская, высокая]"


In [1025]:
summary = (
    shift_equipment_night
    .groupby(["machine_id", "machine_type"], as_index=False)
    .agg(
        required=("position", "count"),
        assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum())
    )
)

# display(summary)

# Добавляем столбец "недостает"
summary["missing"] = summary["required"] - summary["assigned"]

# ---- 1. НЕПОЛНЫЕ БРИГАДЫ (частично укомплектованы)
incomplete = summary[
    (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
].copy()

incomplete

,machine_id,machine_type,required,assigned,missing
2,PM-03,высокая,4,3,1
3,PM-04,высокая,4,3,1


In [1026]:
shift_equipment_night

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
5,2,Ночь,PM-02,плоская,2,6,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
8,2,Ночь,PM-03,высокая,1,7,Печатник,W049
9,2,Ночь,PM-03,высокая,2,6,Печатник,W050


In [1027]:
# Второй проход назначения для укомплектования неполных бригад, только неполные бригады
free_machin_positions, shift_equi = position_assignment(
    shift_equipment_night[shift_equipment_night["machine_id"].isin(incomplete["machine_id"])],
    s="third",
    shift_name="Ночь"
)

# Собираем полное расписание смены обратно
shift_equipment_night.update(shift_equi)
display(shift_equipment_night)

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
5,2,Ночь,PM-02,плоская,2,6,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
8,2,Ночь,PM-03,высокая,1,7,Печатник,W049
9,2,Ночь,PM-03,высокая,2,6,Печатник,W050


In [1007]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_nigth[~shift_workers_day["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


In [1008]:
# Выводим результат с именами работников
result = shift_equipment_day.merge(
    workers[["worker_id", "name"]], on="worker_id", how="left"
)
result

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id,name
0,2,День,PM-01,плоская,1,7,Печатник,W021,Работник П 21
1,2,День,PM-01,плоская,2,6,Печатник,W022,Работник П 22
2,2,День,PM-01,плоская,3,5,Печатник,W023,Работник П 23
3,2,День,PM-01,плоская,4,4,Печатник,W037,Работник О 37
4,2,День,PM-02,плоская,1,7,Печатник,W025,Работник П 25
5,2,День,PM-02,плоская,2,6,Печатник,W026,Работник П 26
6,2,День,PM-02,плоская,3,5,Печатник,W027,Работник П 27
7,2,День,PM-02,плоская,4,4,Печатник,W038,Работник О 38
8,2,День,PM-03,высокая,1,7,Печатник,W029,Работник В 29
9,2,День,PM-03,высокая,2,6,Печатник,W030,Работник В 30


In [ ]:
# Группируем по машинам и позициям, чтобы получить минимальный разряд и требуемые профессии
by_machine_position = result.groupby(
    ["week", "shift", "machine_type", "machine_id", "position"], as_index=True
).agg(
    # machine_type=('machine_type', 'first'),
    # position = ('position', 'first'),
    min_rank=("min_rank", "min"),
    worker_id=("worker_id", "first"),
    name=("name", "first"),
    profession_required=("profession_required", "first"),
)

display(by_machine_position)

In [ ]:
by_machine_position = by_machine_position.reset_index()
by_machine_position

In [ ]:
schedule.head()

In [ ]:
temp = result[["week", "shift", "worker_id"]]
temp